# Cryomagnetics 4G QCoDeS Driver Example

This notebook showcases how to interfacing with a Cryomagnetics 4G power supply using the QCoDeS. The Cryomagnetics 4G power supply is commonly used in research settings for precise magnetic field control in experiments that include superconductivity studies and material science.

## Setup

First, ensure you have the required drivers and QCoDeS installed. The connection to the instrument is assumed to be via GPIB. Let's establish the connection and set up the initial parameters for the magnetic field control.

In [1]:
from qcodes.instrument_drivers.cryomagnetics import CryomagneticsModel4G

max_current_limits = {
    1: (45.0, 0.01),
    2: (67.24, 0.0138),
    3: (85, 0.0001),
    4: (95, 0.0001),
}
coil_constant = 0.13385  # T/A

cryo_instrument = CryomagneticsModel4G(
    name='cryo_4g',
    address='GPIB::1::INSTR',
    max_current_limits=max_current_limits,
    coil_constant=coil_constant,
    pyvisa_sim_file="cryo4g.yaml",
)

Connected to:  None (serial:None, firmware:None) in 0.03s


## Basic Operations

The following sections demonstrate how to perform basic operations with the Cryomagnetics 4G instrument. We will cover setting and measuring the magnetic field, checking the magnet's operational state, and modifying the ramp rate for adjusting the magnetic field.

In [2]:
# Checking the coil constant
coil_constant = cryo_instrument.coil_constant
print(f'Coil constant = {coil_constant} T/A')

Coil constant = 0.13385 T/A


In [3]:
import time

import matplotlib.pyplot as plt


# Function to plot magnetic field over time
def plot_field_over_time(cryo_instr, target, duration):
    times = []
    fields = []
    start_time = time.time()
    while (time.time() - start_time) < duration:
        fields.append(cryo_instr.field())
        times.append(time.time() - start_time)
        time.sleep(0.5)
    plt.plot(times, fields, marker='o')
    plt.xlabel('Time (s)')
    plt.ylabel('Magnetic Field (T)')
    plt.title(f'Time vs Magnetic Field towards {target} T')
    plt.show()

# Example usage:
# plot_field_over_time(cryo_instrument, 1.0, 60)  # plot for 60 seconds while setting the field to 1 T


## Range and Rate Settings

The Cryomagnetics4G.py driver provides functionality to manage the current ranges and ramp rates of the Cryomagnetics 4G superconducting magnet power supply. These settings are crucial for ensuring safe and efficient operation of the magnet within its specified limits.

### Current Ranges

The power supply has multiple current ranges, each with its own upper current limit. The `max_current_limits` parameter in the `CryomagneticsModel4G` class constructor allows you to specify these ranges and their corresponding limits. It is a dictionary where the keys represent the range indices, and the values are tuples containing the upper current limit and maximum rate for that range.

For example:
```python
max_current_limits = {
    1: (45.0, 0.01),
    2: (67.24, 0.0138),
    3: (85, 0.0001),
    4: (95, 0.0001),
}
```

In this example, there are four current ranges defined:
- Range 1: Upper current limit of 45.0 A and a maximum rate of 0.01 A/s.
- Range 2: Upper current limit of 67.24 A and a maximum rate of 0.0138 A/s.
- Range 3: Upper current limit of 85 A and a maximum rate of 0.0001 A/s.
- Range 4: Upper current limit of 95 A and a maximum rate of 0.0001 A/s.

The driver automatically handles the selection of the appropriate range based on the current field value when setting the ramp rate.

### Ramp Rates

The `rate` parameter in the driver represents the ramp rate of the magnetic field in Tesla per minute. When setting the ramp rate, the driver ensures that it does not exceed the maximum rate specified for the corresponding current range.

If an attempt is made to set a ramp rate that is outside the defined rate ranges, a `ValueError` is raised.

### Initializing Range and Rate Settings

During initialization, the driver sets up the current limits and maximum rates for each range on the instrument based on the provided `max_current_limits` parameter. This ensures that the instrument is properly configured with the specified limits.

By carefully defining the current ranges and ramp rates, you can ensure that the magnet operates within its safe limits and optimize its performance based on your specific requirements.

It is important to consult the manufacturer's documentation and specifications when determining the appropriate current limits and ramp rates for your specific magnet and experimental setup.

### Note on Safety

Always ensure the input parameters remain within the safe operating bounds of your instrument. Manual cross-checking and regular maintenance schedules are recommended to avoid equipment damage or accidents.

For further information, consult the official documentation and user manuals provided by the manufacturer. Always prioritize safe operating practices and maintain the instrument according to the recommended guidelines.